# 공공보건의료기관 데이터 분석

- [전국 공공보건의료기관 현황 파악](www.data.go.kr)
- 공공보건의료기관현황.csv : 행정구역별 의료기관정보 O, 인구 정보 X
- 행정구역시군구별_성별인구수.xlsx : 행정구역별 인구 정보 O, 의료기관정보 X
- 두 데이터를 통해 행정구역, 의료기관정보, 인구수 데이터를 조합 

In [10]:
import pandas as pd
import numpy as np

# pandas max width option setting
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

---
### 공공보건의료기관현황 데이터

In [49]:
publicSanityOrg = pd.read_csv('../../공공데이터수업과제/Data/공공보건의료기관현황.csv',
							  index_col=0, encoding='euc_kr')
print(publicSanityOrg.head())

         병원 명  설립형태                  근거 법령     관계 행정기관 관계 공공단체  심평원 요양기관번호  \
연번                                                                           
1    강원도 재활병원   시도립      강원도재활병원설치및운영에관한조례         강원도    해당없음    32200641   
2   강원도 삼척의료원  특수법인  지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원    32100060   
3   강원도 영월의료원  특수법인  지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원    32100078   
4   강원도 원주의료원  특수법인  지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원    32100086   
5   강원도 강릉의료원  특수법인  지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원    32100159   

    종별구분  병상수 소재지 우편번호                           주소                     홈페이지  \
연번                                                                             
1     병원  165    24227      강원도 춘천시 충열로 142번길 24-16            www.grh.or.kr   
2   종합병원  152    25920             강원도 삼척시 오십천로 418        http://ksmc.or.kr   
3   종합병원  214    26234          강원도 영월군 영월읍 중앙1로 59  http://www.youngwol.org   
4   종합병원  237    26448        강원도 원주시 서원대로 387(개운동)  

'주소' 컬럼 데이터를 공백을 기준으로 '시도'와 '구군' 정보로 분리해서 컬럼 생성 후 창원시, 경산시, 천안시 같은 주소를 시도에 맞는 주소 형식으로 바꾼다.

In [18]:
address = pd.DataFrame(publicSanityOrg['주소'])
address = address['주소'].apply(lambda x: x.split()[:2])
address = pd.DataFrame(address.tolist(), columns=['시도', '군구'])
print(address.head())

,시도,군구
0,강원도,춘천시
1,강원도,삼척시
2,강원도,영월군
3,강원도,원주시
4,강원도,강릉시


In [19]:
print(address['시도'].unique())

['강원도' '경기도' '경기' '경남' '창원시' '경상남도' '경상북도' '경산시' '경북' '인천광역시' '대구광역시'
 '전라남도' '대전광역시' '광주광역시' '제주특별자치도' '부산광역시' '전라북도' '충북' '서울특별시' '서울시'
 '부산특별시' '대전시' '충남' '전남' '충청남도' '울산광역시' '전북' '천안시' '충청북도']


In [30]:
print(address[address['시도']=='천안시'])
address.iloc[[209, 210]] = ['충청남도', '천안시']
print(address.iloc[[209, 210]])

In [ ]:
# '시도' 데이터 변경작업 반복처리 함수 
def addrConv(city):
	idx = list(address[address['시도']==city].index)
	if len(idx) == 1:
		address.iloc[idx[0]]

# 창원시, 경산시, 천안시

list(address[address['군구']=='춘천시'].index

경기도를 경기 또는 경기도로 되어있는 것을 정확한 시도 명칭으로 매칭하는데 addr_aliases 리스트의 key와 매칭되는 값이 없으면 lambda 함수로 '시도' 컬럼에서 불러왔던 원래 x 요소값 그대로 '시도' 컬럼에 넣는다.

In [53]:
addr_aliases = {'경기': '경기도', '경남': '경상남도', '경북': '경상북도',
				'충북': '충청북도', '서울시': '서울특별시', '부산특별시': '부산광역시',
				'대전시': '대전광역시', '충남': '충청남도', '전남': '전라남도', '전북': '전라북도'}
address['시도'] = address['시도'].apply(lambda x: addr_aliases.get(x, x))

0       강원도
1       강원도
2       강원도
3       강원도
4       강원도
       ... 
222    충청북도
223    충청북도
224    충청북도
225    충청북도
226    충청북도
Name: 시도, Length: 227, dtype: object

'군구' 컬럼에서 '아란13길' 주소를 '시도', '군구' 컬럼에 맞게 '제주특별자치도', '제주시'로 변경한다

In [ ]:
print(address['군구'].unique())
print(address[address['군구']=='아란13길'])
address.iloc[75] = ['제주특별자치도', '제주시']
print(address.iloc[75])
print(address['군구'].unique())

address DataFrame에 '시도'와 '군구' 데이터를 합친 '시도/군구'와 0으로 셋팅된 '의료기간 수' 컬럼을 추가한다

In [74]:
address['시도/군구'] = address.apply(lambda column: column['시도'] + ' ' + column['군구'], axis=1)
address['의료기간 수'] = 0
address.head()

,시도,군구,시도/군구,의료기간 수
0,강원도,춘천시,강원도 춘천시,0
1,강원도,삼척시,강원도 삼척시,0
2,강원도,영월군,강원도 영월군,0
3,강원도,원주시,강원도 원주시,0
4,강원도,강릉시,강원도 강릉시,0


3개의 컬럼을 그룹화하고 의료기간의 수를 센 컬럼을 포함한 addr_group DataFrame을 생성한다. address DataFrame의 내용이 변경되는 것이 아니다. addr_group DataFrame은 데이터 병합에 사용될 최종 데이터이다.

In [78]:
addr_group = pd.DataFrame(address.groupby(['시도', '군구', '시도/군구'],
						  as_index=False).count())
addr_group = addr_group.set_index('시도/군구')
addr_group.head()

,시도,군구,의료기간 수
시도/군구,,,
강원도 강릉시,강원도,강릉시,4
강원도 동해시,강원도,동해시,1
강원도 삼척시,강원도,삼척시,1
강원도 속초시,강원도,속초시,1
강원도 양구군,강원도,양구군,1


---
### 행정구역별 인구 데이터

In [80]:
# %pip install openpyxl
poplulation = pd.read_excel('../../공공데이터수업과제/Data/행정구역시군구별_성별인구수.xlsx')
poplulation.head()

,행정구역(시군구)별(1),행정구역(시군구)별(2),총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


공공보건의료기관현황.csv 데이터와 합치기 위해 컬럼명을 공통된 이름으로 수정한다. 

In [81]:
poplulation = poplulation.rename(columns={'행정구역(시군구)별(1)': '시도',
										  '행정구역(시군구)별(2)': '군구'})
poplulation.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


'군구' 컬럼 데이터에 공백이 포함되어 있어 공백을 제거 후 공공보건의료기관현황 데이터와 마찬가지로 '시도'와 '군구' 데이터를 합친 '시도/군구' 컬럼을 생성한다.

In [85]:
poplulation['군구'][3]

'\u3000\u3000\u3000중구'

In [87]:
poplulation['군구'] = poplulation['군구'].apply(lambda x: x.strip())
poplulation['군구'][3]

'중구'

In [89]:
poplulation['시도/군구'] = poplulation['시도'] + ' ' + poplulation['군구']
poplulation.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명),시도/군구
0,전국,소계,51847509,25862863,25984646,전국 소계
1,서울특별시,소계,9733509,4745088,4988421,서울특별시 소계
2,서울특별시,종로구,151215,73688,77527,서울특별시 종로구
3,서울특별시,중구,126201,61946,64255,서울특별시 중구
4,서울특별시,용산구,229385,110701,118684,서울특별시 용산구
...,...,...,...,...,...,...
274,경상남도,거창군,62049,30275,31774,경상남도 거창군
275,경상남도,합천군,45054,21590,23464,경상남도 합천군
276,제주특별자치도,소계,670749,337231,333518,제주특별자치도 소계
277,제주특별자치도,제주시,489202,245389,243813,제주특별자치도 제주시


'소계' 데이터 삭제 후 '시도/군구' 컬럼을 인덱스로 설정한다

In [90]:
poplulation = poplulation[poplulation['군구'] != '소계']
poplulation = poplulation.set_index('시도/군구')
poplulation.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명),시도/군구
2,서울특별시,종로구,151215,73688,77527,서울특별시 종로구
3,서울특별시,중구,126201,61946,64255,서울특별시 중구
4,서울특별시,용산구,229385,110701,118684,서울특별시 용산구
5,서울특별시,성동구,300410,147020,153390,서울특별시 성동구
6,서울특별시,광진구,351263,170164,181099,서울특별시 광진구
...,...,...,...,...,...,...
273,경상남도,함양군,39555,19058,20497,경상남도 함양군
274,경상남도,거창군,62049,30275,31774,경상남도 거창군
275,경상남도,합천군,45054,21590,23464,경상남도 합천군
277,제주특별자치도,제주시,489202,245389,243813,제주특별자치도 제주시


---
### '공공보건의료기관현황'과 '행정구역시군구별_성별인구수' 데이터 병합

In [112]:
addr_pop_merge = pd.merge(addr_group, poplulation, left_index=True, right_index=True, how='inner')
addr_pop_merge.head()

,시도_x,군구_x,의료기간 수,시도_y,군구_y,총인구수 (명),남자인구수 (명),여자인구수 (명)
시도/군구,,,,,,,,
강원도 강릉시,강원도,강릉시,4,강원도,강릉시,213328,105775,107553
강원도 동해시,강원도,동해시,1,강원도,동해시,90417,45782,44635
강원도 삼척시,강원도,삼척시,1,강원도,삼척시,66806,33811,32995
강원도 속초시,강원도,속초시,1,강원도,속초시,81840,40470,41370
강원도 양구군,강원도,양구군,1,강원도,양구군,22526,11937,10589
...,...,...,...,...,...,...,...,...
충청북도 단양군,충청북도,단양군,1,충청북도,단양군,29664,15023,14641
충청북도 영동군,충청북도,영동군,1,충청북도,영동군,48523,24060,24463
충청북도 제천시,충청북도,제천시,1,충청북도,제천시,134359,67326,67033


In [ ]:
local_medical_population = addr_pop_merge[['시도_x', '군구_x', '의료기관 수', '총인구수 (명)']]
local_medical_population = local_medical_population.rename({'시도_x': '시도',
														  '군구_x': '군구',
														  '의료기관 수': '공공의료기관수',
														  '총인구수 (명)': '전체 인구(명)'})
local_medical_population.head()

In [ ]:
medical_count = local_medical_population['공공의료기관수']
pop_count = local_medical_population['전체 인구(명)']
local_medical_population['인구대비 의료기관비율'] = medical_count.div(pop_count, axis=0) * 100000
local_medical_population.head()

---
### 공공보건의료기관 현황

- 지역별 공공의료기관수

In [99]:
import matplotlib.pyplot as plt
import platform

from matplotlib import rcParams


plt.style.use('ggplot')
rcParams['axes.unicode_minus']=False
if platform.system() == 'Windows':
	plt.rc('font', family='Malgun Gothic')
else:
	plt.rc('font', family='AppleGothic')

In [ ]:
medical_ratio = local_medical_population[['공공의료기관수']].sort_values(ascending=False)
medical_ratio.plot(kind='bar', rot=90, figsize=(30, 6))

- 인구대비 공공의료기관 비율

In [ ]:
pop_medi_ratio = local_medical_population[['인구대비 의료기관비율']].sort_values(ascending=False)
pop_medi_ratio.plot(kind='bar', rot=90, figsize=(30, 6))